<a href="https://colab.research.google.com/github/DevBhuyan/Image-classification/blob/master/Copy_of_Cats_vs_Dogs_augmented.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
tf.__version__

In [ ]:
class mycallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = {}):
        if logs.get('accuracy') >= 0.95:
            print('Reached 95% accuracy, canceling training')
            self.model.stop_training = True

In [ ]:
callbacks = mycallback()

In [ ]:
import os
import zipfile

In [ ]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O /tmp/cats_and_dogs_filtered.zip

In [ ]:
zfile = '/tmp/cats_and_dogs_filtered.zip'
ref = zipfile.ZipFile(zfile, 'r')
ref.extractall('/images')
ref.close()

In [ ]:
base_dir = '/images/cats_and_dogs_filtered'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [ ]:
train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

print(train_cat_fnames)
print('\n\n\n\n\n\n\n\n\n\n\n\n\n')
print(train_dog_fnames)

print('total cat training images :', len(train_cat_fnames))
print('total dog training images :', len(train_dog_fnames))

print('total cat validation images :', len(os.listdir(validation_cats_dir)))
print('total dog validation images :', len(os.listdir(validation_dogs_dir)))

In [ ]:
model = tf.keras.Sequential([
    keras.layers.Conv2D(16, (3, 3), activation = tf.nn.relu, input_shape = (150, 150, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(32, (3, 3), activation = tf.nn.relu),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation = tf.nn.relu),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation = tf.nn.relu),
    keras.layers.Dense(1, activation = tf.nn.sigmoid)
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = keras.optimizers.RMSprop(lr = 0.001),
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

In [ ]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255, 
                                                             rotation_range=40,
                                                             width_shift_range=0.2,
                                                             height_shift_range=0.2,
                                                             shear_range=0.2,
                                                             zoom_range=0.2,
                                                             horizontal_flip=True,
                                                             fill_mode='nearest')
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255)

train_generator = train_datagen.flow_from_directory(train_dir, 
                                                   batch_size = 20,
                                                   class_mode = 'binary',
                                                   target_size = (150, 150))

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                       batch_size = 20,
                                                       class_mode = 'binary',
                                                       target_size = (150, 150))

In [ ]:
history = model.fit(train_generator,
                   validation_data = validation_generator,
                   steps_per_epoch = 100,
                   epochs = 2,
                   validation_steps = 50,
                   verbose = 1,
                   callbacks = [callbacks])

In [ ]:
# PLOT LOSS AND ACCURACY
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')

# Desired output. Charts with training and validation metrics. No crash :)